# Working with coadd and multiband

### In this notebook we are going to show how to use the catalogs produced by forced photometry at the coadd level

Coadd images are the result of the addition of several images projected into a segmented sky map. 

The sky is divided in tract and patches according to a tessellation model. The size of the individual patches, the pixel size and the sky projection are parameters passed to makeSkyMap.py or makeDiscreteSkyMap.py

```
cca001[0] cd /sps/lsst/dev/lsstprod/hsc/SXDS/output
cca001[0] ls
00991  01052  01059  01318  01382      deepCoadd-results                   deep_makeCoaddTempExp_metadata  schema
00995  01055  01062  01327  config     deepCoadd_forcedPhotCoadd_metadata  jointcal-results
01004  01057  01316  01378  deepCoadd  deep_assembleCoadd_metadata         repositoryCfg.yaml
```

```
cca001[130] cd deepCoadd-results/
cca001[0] ls
HSC-G  HSC-I  HSC-R  HSC-Y  HSC-Z  merged
```

```
cca001[0] cd HSC-I/0/3,4/
cca001[0] ls
calexp-HSC-I-0-3,4.fits    detectMD-HSC-I-0-3,4.boost   measMD-HSC-I-0-3,4.boost
det-HSC-I-0-3,4.fits       forced_src-HSC-I-0-3,4.fits  srcMatch-HSC-I-0-3,4.fits
det_bkgd-HSC-I-0-3,4.fits  meas-HSC-I-0-3,4.fits        srcMatchFull-HSC-I-0-3,4.fits
```

- `0` is the tract number (only one in this case)
- `3,4` is the patch number
- there is one separate directory per patch
- `forced_src-HSC-I-0-3,4.fits` contains the forcedPhotCoadd catalog for the 'HSC-I' filter in patch '3,4'



In [84]:
import lsst.daf.persistence as dafPersist

# Initialize butler
butler = dafPersist.Butler("/sps/lsst/dev/lsstprod/hsc/SXDS/output")

# for the dataid we need to specify the tract, the patch and the filter
dataid_r = {'tract':0, 'filter':'HSC-R', 'patch':'3,4'}
forced_r = butler.get('deepCoadd_forced_src', dataId=dataid_r, immedate=True)

dataid_i = {'tract':0, 'filter':'HSC-I', 'patch':'3,4'}
forced_i = butler.get('deepCoadd_forced_src', dataId=dataid_r, immedate=True)

dataid_g = {'tract':0, 'filter':'HSC-G', 'patch':'3,4'}
forced_g = butler.get('deepCoadd_forced_src', dataId=dataid_r, immedate=True)

# The catalogs corresponding to the 3 filters have all the same length
# this is by definition of the forced photometry
print(len(forced_r), len(forced_i), len(forced_g))

25642 25642 25642


In [13]:
# The catalog's objects are also stored in the same order
print(forced_r[235].get('id'), forced_i[235].get('id'), forced_g[235].get('id'))
print(forced_r[235].get('coord_ra'), forced_i[235].get('coord_ra'), forced_g[235].get('coord_ra'))
print(forced_r[235].get('coord_dec'), forced_i[235].get('coord_dec'), forced_g[235].get('coord_dec'))

429496729877 429496729877 429496729877
0.610344 rad 0.610344 rad 0.610344 rad
-0.0904406 rad -0.0904406 rad -0.0904406 rad


In [85]:
# We need the calibration in order to convert flux to magnitude
# We use comprehension dictionaries 

filters = ['HSC-R', 'HSC-I', 'HSC-G']
dataid = {k: {'tract':0, 'filter':k, 'patch':'3,4'} for k in filters}
print(dataid)
calib = {k: butler.get('deepCoadd_calexp', dataId=dataid[k], immediate=True).getCalib() for k in filters}

{'HSC-R': {'tract': 0, 'filter': 'HSC-R', 'patch': '3,4'}, 'HSC-I': {'tract': 0, 'filter': 'HSC-I', 'patch': '3,4'}, 'HSC-G': {'tract': 0, 'filter': 'HSC-G', 'patch': '3,4'}}


In [86]:
# let's do the same for the forced catalog (we could have used the forced_r, forced_i and forced_g catalogs
# created as the previous step, but we want to be more pythonic)
#
# we also convert immediately to astropy tables
forced = {k: butler.get('deepCoadd_forced_src', dataId=dataid[k], immediate=True).asAstropy() for k in filters}
len(forced['HSC-G'])

In [89]:
import numpy as np

sel = np.full(len(forced['HSC-G']), True, dtype=bool)
#sel = np.full_like(forced['HSC-G'], True)

for f in filters:
    sel &= ((forced[f]['modelfit_CModel_flag'] == False) & (forced[f]['modelfit_CModel_flux'] > 0))
    
m = {}
s = {}
for f in filters:
    m[f], s[f] = calib[f].getMagnitude(forced[f]['modelfit_CModel_flux'][sel], 
                                           forced[f]['modelfit_CModel_fluxSigma'][sel])

len(m['HSC-G'])

23293

In [90]:
print(sel)

[ True False False ..., False False False]


In [91]:
# keep only good rows and add 2 columns to store magnitudes and error on magnitudes
for f in filters:
    forced[f] = forced[f][sel]
    forced[f]['magnitude'] = m[f]
    forced[f]['magnitudeErr'] = s[f]


In [92]:
forced['HSC-G']['magnitude']

25.6498904864
26.1147637551
25.2490990216
24.43468454
26.2177585193
25.7697025932
26.3549012342
26.3530631973
28.1385364247
25.603304176
24.7820673462


In [57]:
schema = forced_i.getSchema()
schema.getOrderedNames()

['id',
 'coord_ra',
 'coord_dec',
 'parent',
 'deblend_nChild',
 'base_TransformedCentroid_x',
 'base_TransformedCentroid_y',
 'base_TransformedCentroid_flag',
 'base_TransformedShape_xx',
 'base_TransformedShape_yy',
 'base_TransformedShape_xy',
 'base_TransformedShape_flag',
 'modelfit_DoubleShapeletPsfApprox_0_xx',
 'modelfit_DoubleShapeletPsfApprox_0_yy',
 'modelfit_DoubleShapeletPsfApprox_0_xy',
 'modelfit_DoubleShapeletPsfApprox_0_x',
 'modelfit_DoubleShapeletPsfApprox_0_y',
 'modelfit_DoubleShapeletPsfApprox_0_0',
 'modelfit_DoubleShapeletPsfApprox_0_1',
 'modelfit_DoubleShapeletPsfApprox_0_2',
 'modelfit_DoubleShapeletPsfApprox_0_3',
 'modelfit_DoubleShapeletPsfApprox_0_4',
 'modelfit_DoubleShapeletPsfApprox_0_5',
 'modelfit_DoubleShapeletPsfApprox_1_xx',
 'modelfit_DoubleShapeletPsfApprox_1_yy',
 'modelfit_DoubleShapeletPsfApprox_1_xy',
 'modelfit_DoubleShapeletPsfApprox_1_x',
 'modelfit_DoubleShapeletPsfApprox_1_y',
 'modelfit_DoubleShapeletPsfApprox_1_0',
 'modelfit_DoubleSh